In [1]:
!pip install transformers datasets torch


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer
from datasets import Dataset
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import torch
from transformers import AutoConfig
import torch.nn as nn

In [3]:
# Define a compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Get predicted class by taking the argmax of logits
    preds = predictions.argmax(axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Return as a dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [4]:
# Load your dataset
data_path = "/kaggle/input/power-tr-train/power-tr-train.tsv"  # Replace with your dataset file path
df = pd.read_csv(data_path, sep='\t')

# Check the distribution of labels
label_counts = df['label'].value_counts()

In [5]:
# Print class distribution
print("Class Distribution:")
for label, count in label_counts.items():
    print(f"Label {label}: {count} ({(count / len(df)) * 100:.2f}%)")


Class Distribution:
Label 1: 8932 (51.38%)
Label 0: 8452 (48.62%)


In [6]:
# Perform stratified split based on the 'label' column
train_df, test_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

print(train_df)
print(test_df)

# Create a Hugging Face Dataset object from the DataFrame
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

            id                           speaker sex  \
7838   tr25984  9f2161636a7c1bbd385e7893bc0d9c85   M   
11440  tr29586  e20ee998a6c5cb6a6b95034dfa2808ba   M   
10808  tr28954  2ead8c44e83722110aeafcda867a8ba3   M   
12133  tr30279  9ea3207b08f5e93042b3b8ef4fa7b7b3   M   
14530  tr32676  2e5b14b14f72f3d24f86401121a0c37a   F   
...        ...                               ...  ..   
15943  tr34089  498dcde0a7f7faee74e604562258ddb3   F   
6269   tr24415  043d8ce9b522d8896d08711df295a5e7   M   
7115   tr25261  d69dee113e92cbf04647c0565ccc4da3   F   
14666  tr32812  1324e52f6dc5a4677e80b2246090910f   M   
9110   tr27256  a71f9077d9a1f752c5dacedc685f635d   M   

                                                    text  \
7838   Değerli arkadaşlar, bakın, bugün enerjide önem...   
11440  Bu bütçeye “evet” demek Kürt emekçilerine “Dil...   
10808  Sayın Başkanım, vekil arkadaşımızın ifade etmi...   
12133  Sayın Başkan, bir cevap verebilir miyim? <p> “...   
14530  Sayın Başkan, değerl

In [7]:
device = torch.device("cuda:0")

# Load pre-trained model and tokenizer
model_name = "xlm-roberta-base"
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Tokenize the datasets(English texts)
def tokenize_function(examples):
    return tokenizer(examples['text_en'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/15645 [00:00<?, ? examples/s]

Map:   0%|          | 0/1739 [00:00<?, ? examples/s]

In [9]:
# Rename columns for the trainer
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

# Remove unnececcary columns (Turkish column is removed)
train_dataset = train_dataset.remove_columns(["id", "text"])
test_dataset = test_dataset.remove_columns(["id", "text"])

print(train_dataset)
print(test_dataset)


Dataset({
    features: ['speaker', 'sex', 'text_en', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 15645
})
Dataset({
    features: ['speaker', 'sex', 'text_en', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 1739
})


In [10]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./power-en-results",
    evaluation_strategy="epoch",  # Evaluation during training
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model after each epoch
    logging_dir='./logs',  # Logging directory
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # Include metric computation
)

In [12]:
# Train the model
trainer.train()

# Save the model after training
trainer.save_model()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.308800,0.361302,0.838988,0.934844,0.738255,0.825000
2,0.275900,0.342198,0.875216,0.920497,0.828859,0.872278
3,0.176900,0.381324,0.890167,0.897175,0.888143,0.892636


In [13]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.34219783544540405, 'eval_accuracy': 0.875215641173088, 'eval_precision': 0.9204968944099379, 'eval_recall': 0.8288590604026845, 'eval_f1': 0.8722778104767511, 'eval_runtime': 31.1908, 'eval_samples_per_second': 55.754, 'eval_steps_per_second': 0.898, 'epoch': 3.0}
